In [1]:
import h5py
import numpy as np
import pandas as pd
from astropy.io import fits
import fitsio

In [2]:
wide_field_file = '/global/cscratch1/sd/acampos/sompz_data/Y3_mastercat_03_31_20.h5'

In [3]:
with h5py.File(wide_field_file,'r') as file:
    coadd = np.array(file['catalog/metacal/unsheared/coadd_object_id'][...])

In [7]:
# f = h5py.File('sompz_test.hdf5','r')

# fluxtypes = ['unsheared', 'sheared_1m', 'sheared_1p', 'sheared_2m', 'sheared_2p']
# fluxtype = fluxtypes[0]

# f[f'catalog/sompz/{fluxtype}'].keys()

# df_dict_cell = {'cell_wide': np.array(f[f'catalog/sompz/{fluxtype}/cell_wide'])}
# df_dict = {}
# df_dict.update(df_dict_cell)
# sompzh5_data = pd.DataFrame(df_dict)

# sompzh5_data['bhat'] = np.array(f[f'catalog/sompz/{fluxtype}/bhat'])
# sompzh5_data['coadd_object_id'] = np.array(f['catalog/sompz/unsheared/coadd_object_id'])


In [8]:
f = h5py.File('sompz_test.hdf5','r')
s = h5py.File('sompz_SOMF_all_601.hdf5','w', track_order=True)

fluxtypes = ['unsheared', 'sheared_1m', 'sheared_1p', 'sheared_2m', 'sheared_2p']

for fluxtype in fluxtypes:
    
    
    f[f'catalog/sompz/{fluxtype}'].keys()
    
    df_dict_cell = {'cell_wide': np.array(f[f'catalog/sompz/{fluxtype}/cell_wide'])}
    df_dict = {}
    df_dict.update(df_dict_cell)
    sompzh5_data = pd.DataFrame(df_dict)

    sompzh5_data['bhat'] = np.array(f[f'catalog/sompz/{fluxtype}/bhat'])
    sompzh5_data['coadd_object_id'] = np.array(f['catalog/sompz/unsheared/coadd_object_id'])
    
    
    sompzh5_data = sompzh5_data.set_index('coadd_object_id')
    
    all_data = pd.DataFrame({'coadd_object_id':coadd})
    all_data['coadd'] = coadd
    all_data = all_data.set_index('coadd_object_id')
    all_data['cell_wide'] = sompzh5_data['cell_wide']
    all_data['bhat'] = sompzh5_data['bhat']
    all_data[np.isnan(all_data)] = -1
    
    if fluxtype == 'unsheared':
        s.create_dataset('catalog/sompz/unsheared/coadd_object_id', data = coadd)
    s.create_dataset(f'catalog/sompz/{fluxtype}/cell_wide', data = all_data['cell_wide'].astype(int).values)
    s.create_dataset(f'catalog/sompz/{fluxtype}/bhat', data = all_data['bhat'].astype(int).values)
    
f.close()    
s.close()    

In [9]:
with fits.open('/global/cscratch1/sd/acampos/2pt_pipeline/y3_final_unblind_test/2pt_fits/2pt_NG.fits') as f:
    f.info()
    data = f['nz_source'].data

Filename: /global/cscratch1/sd/acampos/2pt_pipeline/y3_final_unblind_test/2pt_fits/2pt_NG.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  COVMAT        1 ImageHDU        17   (900, 900)   float64   
  2  xip           1 BinTableHDU     37   200R x 8C   [K, K, K, D, D, D, D, D]   
  3  xim           1 BinTableHDU     37   200R x 8C   [K, K, K, D, D, D, D, D]   
  4  gammat        1 BinTableHDU     37   400R x 8C   [K, K, K, D, D, D, D, D]   
  5  wtheta        1 BinTableHDU     37   100R x 8C   [K, K, K, D, D, D, D, D]   
  6  nz_source     1 BinTableHDU     32   601R x 7C   [D, D, D, D, D, D, D]   
  7  nz_lens       1 BinTableHDU     31   601R x 8C   [D, D, D, D, D, D, D, D]   


In [10]:
# data['Z_HIGH']

In [11]:
s = h5py.File('sompz_SOMF_all_601.hdf5','r+', track_order=True)

s.create_dataset('catalog/sompz/pzdata/zhigh', data = data['Z_HIGH'])
s.create_dataset('catalog/sompz/pzdata/zlow', data = data['Z_LOW'])

s.close()

In [12]:
s = h5py.File('sompz_test.hdf5','r', track_order=True)

In [13]:
s['catalog/sompz/pzdata/pz_chat/'][...]

array([[0.00000000e+00, 0.00000000e+00, 1.02109115e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.85759892e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.61476883e+00, 6.59712267e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.70168789e+00, 1.54497318e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.27336594e+01, 2.98150887e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [14]:
nz=fitsio.read('Y3_y3_redshift_distributions_test_modal_even_Y3_bincond_pileup3.fits')
pz_chat = np.load('pzchat.npy')
pz_c = np.load('pzc.npy')

In [15]:
pz_chat

array([[0.00000000e+00, 0.00000000e+00, 1.02109115e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.85759892e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.61476883e+00, 6.59712267e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.70168789e+00, 1.54497318e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.27336594e+01, 2.98150887e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [16]:
fh = h5py.File('sompz_SOMF_all_601.hdf5','r+', track_order=True)

# f.create_dataset('catalog/sompz/unsheared/coadd_object_id', data = coadd)
# f.create_dataset('catalog/sompz/unsheared/cell_wide', data = cells_wide_unsheared)
# f.create_dataset('catalog/sompz/sheared_1m/cell_wide', data = cells_shared_1m)
# f.create_dataset('catalog/sompz/sheared_1p/cell_wide', data = cells_shared_1p)
# f.create_dataset('catalog/sompz/sheared_2m/cell_wide', data = cells_shared_2m)
# f.create_dataset('catalog/sompz/sheared_2p/cell_wide', data = cells_shared_2p)




fh.create_dataset('catalog/sompz/pzdata/bin0', data = nz['BIN1'])
fh.create_dataset('catalog/sompz/pzdata/bin1', data = nz['BIN2'])
fh.create_dataset('catalog/sompz/pzdata/bin2', data = nz['BIN3'])
fh.create_dataset('catalog/sompz/pzdata/bin3', data = nz['BIN4'])
# fh.create_dataset('catalog/sompz/pzdata/zhigh', data = data['Z_HIGH'])
# fh.create_dataset('catalog/sompz/pzdata/zlow', data = data['Z_LOW'])

fh.create_dataset('catalog/sompz/pzdata/pz_chat', data = pz_chat)
fh.create_dataset('catalog/sompz/pzdata/pz_c', data = pz_c)

fh.close()